# 6.1 Numerical Analysis: Integration and differentiation with libraries

#### Before we start:
* Comments regarding midterm
* Review integration with library end of class 5.2

#### Today's class:

* Derivatives
    - Derivative of numerical data
    - Non-equidistant and noisy data
    - Gradient of 2D function
* Miscellaneous
    - `map`
    - Multi-threaded processing
    - Animation, make movie with ffmpeg


In [ ]:
%pylab ipympl

## Derivatives
We want to take a closer look at derivatives, both from analytical function, from noisy data, from unevenly-spaced data and the gradient of 2D functions. We want to start using libraries for these tasks. 

Let's start with the polynomial `fpoly` first introduduced in class 4.2. There we developed a 2nd-order accurate function for the derivative, `deriv2`.


In [ ]:
def fpoly(x,a=-1.,b=1.,c=1.,d=1.):
    '''Return 3rd-order polynomial
    '''
    ff = a*x**3 + b*x**2 + c*x + d
    return ff

In [ ]:
def fpoly_deriv(x,a=-1.,b=1.,c=1.,d=1.):
    '''Return derivative of 3rd-order polynomial
    '''
    ff = 3.*a*x**2 + 2*b*x + c
    return ff

In [ ]:
def deriv2(func,x,h):
    dfdx = (func(x+h) - func(x-h)) / (2*h)
    return dfdx

Compare one more time numerical and analytical derivative:

In [ ]:
deriv2(fpoly,-0.5,0.1)

In [ ]:
fpoly_deriv(-0.5)

As it may be expected there is of course a library that essentially does the same as our `dervi2`, and a little more. Let's have a look at the doc string.

In [ ]:
from scipy import misc as sm

In [ ]:
#sm.derivative?

In [ ]:
sm.derivative(fpoly,-0.5,dx=0.1,n=1,order=3)

In [ ]:
x = linspace(-2,2,101)
ifig=11;close(ifig);figure(ifig)
plot(x,fpoly(x),label="$f$")
plot(x,sm.derivative(fpoly,x,dx=0.001,n=1,order=3),'--',label="$f'$")
plot(x,sm.derivative(fpoly,x,dx=0.001,n=2,order=5),'-.',label="$f''$")
plot(x,sm.derivative(fpoly,x,dx=0.001,n=3,order=7),':',label="$f'''$")
legend()

### Derivative of numerical data
There are a couple other perfectly valid approaches. One it the numpy method `gradient`.


In [ ]:
y = fpoly(x)

This is one way to calculate the centre-valued derivative:

In [ ]:
yy=(y[2:]-y[0:-2])/(x[2]-x[0])

In [ ]:
diff(y)[0:2]

The numpy `gradient` method is as work horse that will also calculate gradient of multi-dimensional data.

In [ ]:
# gradient?

In [ ]:
ifig=10;close(ifig);figure(ifig)
plot(x,y,label="$f$")
plot(x[1:-1],yy,'-.',label="$f'$")
plot(x[0:-1]+0.5*diff(x)[0],diff(y)/diff(x),'--',label="diff $f'$")
plot(x,gradient(y,x,edge_order=2),':',label="gradient $f'$")
legend()

What if the data points are not equidistant?

In [ ]:
random.rand(20)

Note how the random data in $[0 \dots 1]$ needs to be stretched and shifted to cover the desired range. Also, the random data needs to be sorted or the gradient library will be unhappy.

In [ ]:
x = 4*random.rand(10) - 2
x.sort()
y = fpoly(x)

In [ ]:
ifig=12;close(ifig);figure(ifig)
plot(x,y,'o-',label="$f$")
plot(x[0:-1]+0.5*diff(x),diff(y)/diff(x),'--',label="diff $f'$")
plot(x,gradient(y,x,edge_order=1),':',label="gradient $f'$")
plot(x,fpoly_deriv(x),'-.',label="fpoly_deriv $f'$")
legend()

Explore the `edge_order=1` option in the gradient method, look at the doc string! What is the effect on the edge values?

### Noisy data
What if the data is noisy?

In [ ]:
x = linspace(-2,2,51)

If your data is noisy in both x and y direction you would have to sort the data in x direction first. Let's keep `x_eps = 0.` for now.

In [ ]:
# let's add some noise to this analytical data
y_eps=1.7; x_eps = 0.
d_err = y_eps*(rand(len(x))-0.5)
y_noise = fpoly(x)+d_err
x_noise = x+x_eps*d_err

In [ ]:
ifig=13;close(ifig);figure(ifig)

plot(x_noise,y_noise,'r-o',label='noisy polynomial')
plot(x_noise,gradient(y_noise,x_noise),':',label="gradient $f'$")

legend(loc=0)
xlabel('x')
ylabel('f(x)')

In order to differentiate this data we may first want to smooth and/or interpolate it.


In [ ]:
from scipy import interpolate

In [ ]:
# interpolate.splrep?

In [ ]:
ifig=16;close(ifig);figure(ifig)

plot(x_noise,fpoly(x_noise),'-',label='polynomial')
plot(x_noise,y_noise,'o',label='noisy polynomial')

# create spline representation
tck = interpolate.splrep(x_noise,y_noise, s=15)
x_new = linspace(-1.95,1.95,4*len(x_noise))

# evaluate spline representation
y_new = interpolate.splev(x_new, tck, der=0)
plot(x_new,y_new,'-.',label='smoothed spline $f$')

# once you have a spline representation you can also evaluate the 
# second derivative
yy_new = interpolate.splev(x_new, tck, der=1)
plot(x_new,yy_new,':',label="smoothed spline  $f'$")
plot(x_noise,fpoly_deriv(x_noise),'--',label='fpoly_deriv')


legend(loc=0)
xlabel('x')
ylabel('f(x)')

### Gradient of 2D function

We can use the gradient function for multi-dimensional arrays as well. 

In [ ]:
%pylab ipympl

In [ ]:
c=1; b=0
ff = lambda x,y: exp(-(x-b)**2 / (2*c**2)) *exp(-(y-b)**2 / (2*c**2))

In [ ]:
n = 25   # change to 20 for real plot
y=x=linspace(-1.5,1.5,n+1)
xv,yv = meshgrid(x,y)
z= ff(xv,yv)

In [ ]:
shape(z)

In [ ]:
ifig = 10; close(ifig); figure(ifig)
imshow(z)

In [ ]:
close(22);fig = plt.figure(22)
CS = plt.contour(xv, yv, z,cmap='twilight')
plt.clabel(CS, inline=1, fontsize=8)

Now we calculate the gradient. We should inspect the output.

In [ ]:
shape(gradient(z))

In [ ]:
ifig = 11; close(ifig); figure(ifig)
imshow(gradient(z)[0])  # the first array is the derivative in the y direction

The `quiver` plot is exactly what we need to visualize the gradient.

In [ ]:
ifig = 12; close(ifig); figure(ifig)
quiver(x,y,gradient(z)[1],gradient(z)[0])

In [ ]:
gpx,gpy = gradient(z)[1],gradient(z)[0]
gp_abs = sqrt(gpx**2+gpy**2)

In [ ]:
amax(gp_abs)

In [ ]:
i,j = where(gp_abs > 0.072)

In [ ]:
plot(x[i],y[j],'ro')

## Integration
* see end of nb 5.2

## Miscellaneous

### map
[map](https://docs.python.org/3/library/functions.html#map)
is one of the [built-in Python functions](https://docs.python.org/3/library/functions.html) 
[map](https://www.geeksforgeeks.org/python-map-function/). It returns an iterator that applies function to every item of iterable, yielding the results. `map` in combination with the `lambda` function provides a very similar (if not the same) functionality as list comprehension.

**Example 1:**

In [ ]:
my_pets = ['alfred', 'tabitha', 'william', 'arla']

In [ ]:
my_pets[0].upper()

In [ ]:
[pet.upper() for pet in my_pets]

In [ ]:
list(map(str.upper, my_pets))

**Example 2:**

In [ ]:
f = lambda x,y,z: 100*x + 10*y + z
f(1,2,3)

In [ ]:
x=linspace(1,3,3)
y=linspace(4,6,3)
z=linspace(7,9,3)
print(x,y,z)

In [ ]:
xx = 1
f(xx,y,z)

In [ ]:
[f(xx,y,z) for xx in range(1,3)]

In [ ]:
f_ = lambda x: f(xx,y,z)

In [ ]:
[f_(x) for x in range(1,3)]

In [ ]:
cc = map(f_,range(2,4)) 

In [ ]:
cc?

In [ ]:
array(list(cc))

**Example 3:** 

In [ ]:
C = [39.2, 36.5, 37.3, 38, 37.8] 
F = list(map(lambda x: (float(9)/5)*x + 32, C))
print("{:7s}  {:10s}".format('Celsius','Fahrenheit'))
for a,b, in zip(C,F): print("{:7.2f}  {:10.2f}".format(a,b))

### Multiprocessing
In lecture 5.2 we saw that the MC integration, if done for millions of runs, starts to use measurable computing time. This is an example for a case where a particular task needs to be done over and over again, maybe in another situation with different input parameters. Such problems are called _embarissingly parallel_, essentially because it takes no or very little effort to parallelize them. The following introduces a multiprocessing approach that will use multiple cpu cores (see notebook 3.1 where we covered the hardware aspects). 

The following very simple code example demonstrates the use of the [multiprocessing](https://docs.python.org/2/library/multiprocessing.html) module.

Note: Using the `pool` functionality requires that the `__main__` module be importable by the children. This means that `pool` will not work properly in an interactive session. Thus, the following would have to be used as a Python script:

```Python
from multiprocessing import Pool

def f(x):
    return x*x

p = Pool(5)
print(p.map(f, [1, 2, 3]))
```

In [ ]:
def f(x):
    return x*x

print(list(map(f, [1, 2, 3])))

#### Example MC integration

In [ ]:
%pylab ipympl

In [ ]:
def mcint(func,xrange,n):
    '''MC integration of function func over xrange'''
    dx = diff(xrange)[0]
    x = dx*random.rand(n)+xrange[0]
    favg = func(x).mean()
    I = favg * dx
    return I

In [ ]:
func = lambda x: sin(1/(x*(2-x)))**2

In [ ]:
xrange=(0,2); n = 1001   # nmax = 1e6
x = linspace(*xrange,n)
ifig=1; close(ifig);figure(ifig)
plot(x,func(x))

In [ ]:
mcint(func,xrange,n)

In [ ]:
%%time
ints2 = []
nruns = int(5e5)
nmc=1000
for n in range(nruns):
    ints2.append(mcint(func,xrange,nmc))
print("Mean: {:4.2f}  Variance: {:4.2f}".format(mean(ints2),var(ints2)))

`nruns` determines how many times we are going to determine the integral with a complete MC integration. Remember, each time we will get a slightly different answer because each time a different set of random numbers will be drawn. Each MC integration will use `nmc` random numbers. 

As we can see, doing a million MC integrals takes about a minute. However, we are only using one of our 12 cores. We can use some more with the `multiporcessing` package. 

```Python
from multiprocessing import Pool
import numpy as np

nruns = int(5e6)

def mc(i):
    '''MC integration of function func over xrange'''
    n = 1000
    dx = 2.
    x = dx*np.random.rand(n)
    y = np.sin(1/(x*(2-x)))**2
    favg = y.mean()
    I = favg * dx
    return I


p = Pool(20)
ints3 = np.array(p.map(mc,range(nruns)))
print(ints3.mean(),ints3.var())
```

Run the above program and compare the performance:

In [ ]:
# single process
34.5/nruns

In [ ]:
# 4 procs
4* 74 / 5e6 

In [ ]:
# 10 procs
10* 44 / 5e6 

### Animation, make movie with ffmpeg

A really easy way to make an animation is to generate a sequence of numbered `.png` images and connect them with the command-line program ``ffmpeg`. 

#### Example: Oscillating sine 
We want to make a movie of an oscillating spline.

In [ ]:
ifig=2
close(ifig)
figure(ifig)

In [ ]:
x = linspace(0,2*pi)
ff = 0.25
plot(x, sin(ff*2*pi)*sin(x),lw=0.8)
ylabel('$\sin x$'), xlabel('$x$')

In [ ]:
ifig=3;close(ifig);figure(ifig)
x = linspace(0,2*pi)
fact = linspace(0,2.,32)
for ff in fact:
    plot(x, sin(ff*pi)*sin(x),lw=0.8)
    ylabel('$\sin x$'), xlabel('$x$')

In [ ]:
color=['c', 'm', 'y', 'k','r','g','b']
linestyle=['-', '--', ':', '-.']
cl = [a+b for a in linestyle for b in color]

In [ ]:
from random import shuffle
shuffle(cl)

In [ ]:
ifig=4;close(ifig);figure(ifig)
x = linspace(0,2*pi)
fact = linspace(0,2.,32)
for i,ff in enumerate(fact):
    plot(x, sin(ff*pi)*sin(x),cl[mod(i,len(cl))],lw=0.8)
    ylabel('$\sin x$'), xlabel('$x$')

With the following command the plot can be saved into a png image file:

In [ ]:
savefig('my_great_sin_plot.png')

Now, make one plot per line and save the plot each time as a png image file. 

In [ ]:
ifig=4;close(ifig);figure(ifig)
x = linspace(0,2*pi)
fact = linspace(0,2.,40)
for i,ff in enumerate(fact):
    ifig=4;close(ifig);figure(ifig)
    plot(x, sin(ff*pi)*sin(x),'r-',lw=0.8)
    ylim(-1.1,1.1)
    savefig('sin'+str(i).zfill(4)+".png")
    ylabel('$\sin x$'), xlabel('$x$')

In [ ]:
!rm sin*png

In [ ]:
!ls sin*png

We can use the ffmpeg command to generate out of these frames a movie:
```
#!/bin/bash
# Produces mp4 movie from series of images (e.g. png format)
# ARG1: name base of image files that is followed by numbers, and prefix.
#       for example if the images are named img0001.png than ARG1 is "img"

ffmpeg  -framerate 10   -y -f image2  -pattern_type glob -i "$1*.png" -preset slow -crf 18  -c:v libx264 -b:v 12000k  -pix_fmt yuv420p  $1.mp4

```

In [ ]:
!cat ./bin/movie.sh

In [ ]:
%%bash
./bin/movie.sh sin

This great, but the movie does not look good yet. We need more frames for better time resolution. The notebook environment is not suitable for this, and we can only make one plot at a time. So, let's use multiprocessing again. The python script would look like this:

```Python
from multiprocessing import Pool
from matplotlib import pyplot as pl
import numpy as np

x = np.linspace(0,2*np.pi)
fact = np.linspace(0,5,2000)

def make_image(inp = (1,0.5)):
    i,ff = inp
    pl.plot(x, np.sin(ff*np.pi)*np.sin(x),'r-',lw=0.8)
    pl.ylabel('$\sin x$'), pl.xlabel('$x$')
    pl.ylim(-1.05,1.05)
    pl.savefig('sin'+str(i).zfill(4)+".png")

p = Pool(10)
p.map(make_image,enumerate(fact))
```

Let's try it and make a nice movie ...